<a href="https://colab.research.google.com/github/landcelita/3D-cellular-automaton/blob/main/LBMNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare

In [ ]:
!pip install pygrib
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%load_ext tensorboard

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import os
import time
import datetime as dt
import pygrib
from datetime import timedelta
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.multiprocessing as multiprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# Const

In [ ]:
U_COEF = 0.005
RHO_COEF = 0.0001
U_MSE_COEF = 40000.
RHO_MSE_COEF = 200.
RANDOM_WIDTH = 0.01
HEIGHT = 505
WIDTH = 481

# Define custom layers

### Input Layer

In [ ]:
class InputLayer(nn.Module):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        
    def forward(self, u_vert, u_hori, rho):
        # use feq as input field
        # todo: make this layer be also learnable
        
        # feq(x,v) = C[v] * rho * (1 + 3 v.u + 4.5 (v.u)^2 - 1.5 u^2)
        feq = torch.empty((3, 3, self.height, self.width), dtype=torch.float32, device=device)
        C = torch.tensor([[1.0/36.0, 1.0/9.0, 1.0/36.0], [1.0/9.0, 4.0/9.0, 1.0/9.0], [1.0/36.0, 1.0/9.0, 1.0/36.0]], dtype=torch.float32, device=device)
        u_squared = u_vert ** 2 + u_hori ** 2
        
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                v_dot_u = dr * u_vert + dc * u_hori
                feq[dr+1, dc+1, :, :] = C[dr+1,dc+1] * rho * (1.0 + 3.0 * v_dot_u + 4.5 * v_dot_u * v_dot_u - 1.5 * u_squared)
                
        return feq

In [ ]:
# test for InputLayer
# # Later, I'll check the immutability of the InputLayer
#  - it must put out the same tensor even after the weights are updated
def test_for_InputLayer():
    input_layer_u_vert = torch.tensor([[0.1]], dtype=torch.float32, device=device)
    input_layer_u_hori = torch.tensor([[-0.3]], dtype=torch.float32, device=device)
    input_layer_rho = torch.tensor([[0.8]], dtype=torch.float32, device=device)

    input_layer = InputLayer(1, 1).to(device)
    feq = input_layer(input_layer_u_vert, input_layer_u_hori, input_layer_rho)
    print(torch.sum(feq[2,:,0,0] - feq[0,:,0,0]) / torch.sum(feq[:,:,0,0])) # u_vert
    print(torch.sum(feq[:,2,0,0] - feq[:,0,0,0]) / torch.sum(feq[:,:,0,0])) # u_hori
    print(torch.sum(feq[:,:,0,0])) # rho

test_for_InputLayer()

tensor(0.1000, device='cuda:0')
tensor(-0.3000, device='cuda:0')
tensor(0.8000, device='cuda:0')


### Streaming Layer

In [ ]:
class StreamingLayer(nn.Module):
    def __init__(self, in_height, in_width):
        super().__init__()
        self.in_height = in_height
        self.in_width = in_width
        
        self.w0 = nn.Parameter(torch.zeros((in_height-2, in_width-2), dtype=torch.float32, device=device))
        self.w1 = nn.Parameter(torch.ones((in_height-2, in_width-2), dtype=torch.float32, device=device))
        
    def forward(self, f_prev):
        # f(x,v) = w0(x,v) + w1(x,v) * f_prev(x-v,v)
        
        f = torch.empty((3, 3, self.in_height-2, self.in_width-2), dtype=torch.float32, device=device)
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                f[dr+1,dc+1,:,:] = self.w0 + self.w1 * f_prev[dr+1,dc+1,1-dr:self.in_height-1-dr,1-dc:self.in_width-1-dc]
        
        return f
                

In [ ]:
class StreamingLayerWithNoisyInit(nn.Module):
    def __init__(self, in_height, in_width):
        super().__init__()
        self.in_height = in_height
        self.in_width = in_width
        
        self.w0 = nn.Parameter(torch.zeros((in_height-2, in_width-2), dtype=torch.float32, device=device)\
                               + torch.rand((in_height-2, in_width-2), dtype=torch.float32, device=device) * RANDOM_WIDTH - RANDOM_WIDTH/2.0)
        self.w1 = nn.Parameter(torch.ones((in_height-2, in_width-2), dtype=torch.float32, device=device)\
                               + torch.rand((in_height-2, in_width-2), dtype=torch.float32, device=device) * RANDOM_WIDTH - RANDOM_WIDTH/2.0)
        
    def forward(self, f_prev):
        # f(x,v) = w0(x,v) + w1(x,v) * f_prev(x-v,v)
        
        f = torch.empty((3, 3, self.in_height-2, self.in_width-2), dtype=torch.float32, device=device)
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                f[dr+1,dc+1,:,:] = self.w0 + self.w1 * f_prev[dr+1,dc+1,1-dr:self.in_height-1-dr,1-dc:self.in_width-1-dc]
        
        return f

In [ ]:
# test for StreamingLayer
def test_for_StreamingLayer():
    input_f_prev = torch.zeros((3, 3, 3, 3), dtype=torch.float64, device=device)
    input_f_prev[2,2,0,0] = 1.0
    input_f_prev[2,1,0,1] = 2.0
    input_f_prev[2,0,0,2] = 3.0
    input_f_prev[1,2,1,0] = 4.0
    input_f_prev[1,1,1,1] = 5.0
    input_f_prev[1,0,1,2] = 6.0
    input_f_prev[0,2,2,0] = 7.0
    input_f_prev[0,1,2,1] = 8.0
    input_f_prev[0,0,2,2] = 9.0
    print(input_f_prev)

    streaming_layer = StreamingLayer(3, 3).to(device)
    f_streamed = streaming_layer(input_f_prev)
    print(f_streamed)

test_for_StreamingLayer()

tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 9.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 8., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [7., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 6.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 5., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [4., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 3.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 2., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]], device='cuda:0', dtype=torch.float64)
tensor([[[[9.]],

         [[8.]],

         [[7.]]],


        [[[6.]],

         [[5.]],

         [[4.]]],


        [[[3.]],

         [[2.]],

         [[1.]]]], device='cuda:0', grad_fn=<CopySlices>)


### Colliding Layer

In [ ]:
class CollidingLayer(nn.Module):
    def __init__(self, in_height, in_width):
        super().__init__()
        self.in_height = in_height
        self.in_width = in_width
        
        self.w1 = nn.Parameter(torch.full((in_height, in_width), fill_value=3.0, dtype=torch.float32, device=device), requires_grad=True)
        self.w2 = nn.Parameter(torch.full((in_height, in_width), fill_value=0.0, dtype=torch.float32, device=device), requires_grad=True)
        self.w3 = nn.Parameter(torch.full((in_height, in_width), fill_value=4.5, dtype=torch.float32, device=device), requires_grad=True)
        self.w4 = nn.Parameter(torch.full((in_height, in_width), fill_value=-1.5, dtype=torch.float32, device=device), requires_grad=True)
        
    def forward(self, f_prev):
        # feq(x,v) = C(v) * rho(x) * (1 + w1(x,v) v.u(x) + w2(x,v) vXu(x) + w3(x,v) (v.u(x))^2 + w4(x,v) u(x)^2)
        # f = (f_prev + feq) / 2
        C = torch.tensor([[1.0/36.0, 1.0/9.0, 1.0/36.0], [1.0/9.0, 4.0/9.0, 1.0/9.0], [1.0/36.0, 1.0/9.0, 1.0/36.0]], dtype=torch.float32, device=device)
        
        rho = torch.sum(f_prev, dim=(0,1))
        u_vert = (f_prev[2,0,:,:] + f_prev[2,1,:,:] + f_prev[2,2,:,:] - f_prev[0,0,:,:] - f_prev[0,1,:,:] - f_prev[0,2,:,:]) / rho
        u_hori = (f_prev[0,2,:,:] + f_prev[1,2,:,:] + f_prev[2,2,:,:] - f_prev[0,0,:,:] - f_prev[1,0,:,:] - f_prev[2,0,:,:]) / rho
        u_squared = u_vert ** 2 + u_hori ** 2
        feq = torch.empty((3, 3, self.in_height, self.in_width), dtype=torch.float32, device=device)
        
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                v_dot_u = dr * u_vert + dc * u_hori
                vXu = dr * u_hori - dc * u_vert
                feq[dr+1, dc+1, :, :] = C[dr+1,dc+1] * rho * (1.0 + (self.w1 + self.w3 * v_dot_u) * v_dot_u + self.w2 * vXu + self.w4 * u_squared)
        
        return (f_prev + feq) / 2.0

In [ ]:
class CollidingLayerWithNoisyInit(nn.Module):
    def __init__(self, in_height, in_width):
        super().__init__()
        self.in_height = in_height
        self.in_width = in_width
        
        self.w1 = nn.Parameter(torch.full((in_height, in_width), fill_value=3.0, dtype=torch.float32, device=device)\
                               + torch.rand((in_height, in_width), dtype=torch.float32, device=device) * RANDOM_WIDTH - RANDOM_WIDTH/2.0, requires_grad=True)
        self.w2 = nn.Parameter(torch.full((in_height, in_width), fill_value=0.0, dtype=torch.float32, device=device)\
                               + torch.rand((in_height, in_width), dtype=torch.float32, device=device) * RANDOM_WIDTH - RANDOM_WIDTH/2.0, requires_grad=True)
        self.w3 = nn.Parameter(torch.full((in_height, in_width), fill_value=4.5, dtype=torch.float32, device=device)\
                               + torch.rand((in_height, in_width), dtype=torch.float32, device=device) * RANDOM_WIDTH - RANDOM_WIDTH/2.0, requires_grad=True)
        self.w4 = nn.Parameter(torch.full((in_height, in_width), fill_value=-1.5, dtype=torch.float32, device=device)\
                               + torch.rand((in_height, in_width), dtype=torch.float32, device=device) * RANDOM_WIDTH - RANDOM_WIDTH/2.0, requires_grad=True)
        self.w5 = nn.Parameter(torch.full((in_height, in_width), fill_value=1.0, dtype=torch.float32, device=device)\
                               + torch.rand((in_height, in_width), dtype=torch.float32, device=device) * RANDOM_WIDTH - RANDOM_WIDTH/2.0, requires_grad=True)
        
    def forward(self, f_prev):
        # feq(x,v) = C(v) * rho(x) * (1 + w1(x,v) v.u(x) + w2(x,v) vXu(x) + w3(x,v) (v.u(x))^2 + w4(x,v) u(x)^2)
        # f = (f_prev + feq) / 2
        C = torch.tensor([[1.0/36.0, 1.0/9.0, 1.0/36.0], [1.0/9.0, 4.0/9.0, 1.0/9.0], [1.0/36.0, 1.0/9.0, 1.0/36.0]], dtype=torch.float32, device=device)
        
        rho = torch.sum(f_prev, dim=(0,1))
        u_vert = (f_prev[2,0,:,:] + f_prev[2,1,:,:] + f_prev[2,2,:,:] - f_prev[0,0,:,:] - f_prev[0,1,:,:] - f_prev[0,2,:,:]) / rho
        u_hori = (f_prev[0,2,:,:] + f_prev[1,2,:,:] + f_prev[2,2,:,:] - f_prev[0,0,:,:] - f_prev[1,0,:,:] - f_prev[2,0,:,:]) / rho
        u_squared = u_vert ** 2 + u_hori ** 2
        feq = torch.empty((3, 3, self.in_height, self.in_width), dtype=torch.float32, device=device)
        
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                v_dot_u = dr * u_vert + dc * u_hori
                vXu = dr * u_hori - dc * u_vert
                feq[dr+1, dc+1, :, :] = C[dr+1,dc+1] * rho * (1.0 + (self.w1 + self.w3 * v_dot_u) * v_dot_u + self.w2 * vXu + self.w4 * u_squared)
        
        return (f_prev * self.w5 + feq * (2.0 - self.w5)) / 2.0

In [ ]:
# test for CollidingLayer
def test_for_CollidingLayer():
    input_f = torch.tensor([[[[9., 9.]],[[8., 8.]],[[7., 7.]]],[[[6., 6.]],[[5., 5.]],[[4., 4.]]],[[[3., 3.]],[[2., 2.]],[[1., 1.]]]], dtype=torch.float32, device=device)
    colliding_layer = CollidingLayer(1, 2)
    collided_field = colliding_layer(input_f)

    print(torch.sum(collided_field[:,:,0,0])) # rho 45.0
    print((torch.sum(collided_field[2,:,0,0]) - torch.sum(collided_field[0,:,0,0])) / torch.sum(collided_field[:,:,0,0])) # u_vert
    print((1.+2.+3.-9.-8.-7.)/45.)
    print((torch.sum(collided_field[:,2,0,0]) - torch.sum(collided_field[:,0,0,0])) / torch.sum(collided_field[:,:,0,0])) # u_hori
    print((1.+4.+7.-3.-6.-9.)/45.)

test_for_CollidingLayer()

tensor(45., device='cuda:0', grad_fn=<SumBackward0>)
tensor(-0.4000, device='cuda:0', grad_fn=<DivBackward0>)
-0.4
tensor(-0.1333, device='cuda:0', grad_fn=<DivBackward0>)
-0.13333333333333333


### Output Layer

In [ ]:
class OutputLayer(nn.Module):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        
    def forward(self, f):
        rho = torch.sum(f, dim=(0,1))
        u_vert = (f[2,0,:,:] + f[2,1,:,:] + f[2,2,:,:] - f[0,0,:,:] - f[0,1,:,:] - f[0,2,:,:]) / rho
        u_hori = (f[0,2,:,:] + f[1,2,:,:] + f[2,2,:,:] - f[0,0,:,:] - f[1,0,:,:] - f[2,0,:,:]) / rho
        return {
            'u_vert': u_vert,
            'u_hori': u_hori,
            'rho': rho
        }

# Construct a LBM model from the layers

In [ ]:
class LBM(nn.Module):
    margin = 4
    
    def __init__(self, height, width):
        super(LBM, self).__init__()
        self.input_layer = InputLayer(height, width)
        self.streaming1_layer = StreamingLayer(height, width)
        self.colliding1_layer = CollidingLayer(height-2, width-2)
        self.streaming2_layer = StreamingLayer(height-2, width-2)
        self.colliding2_layer = CollidingLayer(height-4, width-4)
        self.streaming3_layer = StreamingLayer(height-4, width-4)
        self.colliding3_layer = CollidingLayer(height-6, width-6)
        self.streaming4_layer = StreamingLayer(height-6, width-6)
        self.output_layer = OutputLayer(height-8, width-8)
        
    def forward(self, u_vert, u_hori, rho):
        f = self.input_layer(u_vert, u_hori, rho)
        f = self.streaming1_layer(f)
        f = self.colliding1_layer(f)
        f = self.streaming2_layer(f)
        f = self.colliding2_layer(f)
        f = self.streaming3_layer(f)
        f = self.colliding3_layer(f)
        f = self.streaming4_layer(f)
        out = self.output_layer(f)
        return out

In [ ]:
class LBMwithNoisyInit(nn.Module):
    margin = 5
    
    def __init__(self, height, width):
        super(LBMwithNoisyInit, self).__init__()
        self.input_layer = InputLayer(height, width)
        self.streaming1_layer = StreamingLayerWithNoisyInit(height, width)
        self.colliding1_layer = CollidingLayerWithNoisyInit(height-2, width-2)
        self.streaming2_layer = StreamingLayerWithNoisyInit(height-2, width-2)
        self.colliding2_layer = CollidingLayerWithNoisyInit(height-4, width-4)
        self.streaming3_layer = StreamingLayerWithNoisyInit(height-4, width-4)
        self.colliding3_layer = CollidingLayerWithNoisyInit(height-6, width-6)
        self.streaming4_layer = StreamingLayerWithNoisyInit(height-6, width-6)
        self.colliding4_layer = CollidingLayerWithNoisyInit(height-8, width-8)
        self.streaming5_layer = StreamingLayerWithNoisyInit(height-8, width-8)
        self.output_layer = OutputLayer(height-10, width-10)
        
    def forward(self, u_vert, u_hori, rho):
        f = self.input_layer(u_vert, u_hori, rho)
        f = self.streaming1_layer(f)
        f = self.colliding1_layer(f)
        f = self.streaming2_layer(f)
        f = self.colliding2_layer(f)
        f = self.streaming3_layer(f)
        f = self.colliding3_layer(f)
        f = self.streaming4_layer(f)
        f = self.colliding4_layer(f)
        f = self.streaming5_layer(f)
        out = self.output_layer(f)
        return out

# Load data to make a dataset and dataloader

### Transformers

In [ ]:
def transform(u_vert, u_hori, rho):
    # 下向き正
    return u_vert * -U_COEF, u_hori * U_COEF, rho * RHO_COEF

def target_transform(u_vert, u_hori, rho, margin):
    if margin == 0:
        return u_vert * -U_COEF, u_hori * U_COEF, rho * RHO_COEF
    else:
        return u_vert[margin:-margin, margin:-margin] * -U_COEF, u_hori[margin:-margin, margin:-margin] * U_COEF, rho[margin:-margin, margin:-margin] * RHO_COEF

def remove_margin(u_vert, u_hori, rho, margin):
    if margin == 0:
        return u_vert, u_hori, rho
    else:
        return u_vert[margin:-margin, margin:-margin], u_hori[margin:-margin, margin:-margin], rho[margin:-margin,margin:-margin]

### Dataset

In [ ]:
class WeatherDatasetLazy(torch.utils.data.Dataset):
    def __init__(self, src_dir, datetimes, transform, target_transform, margin):
        self.src_dir = src_dir
        self.input_paths = []
        self.target_paths = []
        self.datetimes = []
        self.margin = margin # this value is the number of streaming layers that reduce the border cells.
        # eg. ...                            xxx
        #     ...  --(1 Streaming Layer)-->  x.x
        #     ...                            xxx
        for datetime in datetimes:
            self.datetimes.append(datetime.strftime('%Y%m%d%H'))
            self.input_paths.append(os.path.join(src_dir, datetime.strftime('%Y%m%d%H.grib2')))
            datetime += timedelta(hours=3)
            self.target_paths.append(os.path.join(src_dir, datetime.strftime('%Y%m%d%H.grib2')))
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.input_paths)
    
    def __getitem__(self, idx):
        datetime = self.datetimes[idx]
        input_grib = pygrib.open(self.input_paths[idx])
        rho = torch.from_numpy(np.array(input_grib.select()[0].values, dtype=np.float32)).to(device)
        u_hori = torch.from_numpy(np.array(input_grib.select()[1].values, dtype=np.float32)).to(device)
        u_vert = torch.from_numpy(np.array(input_grib.select()[2].values, dtype=np.float32)).to(device)
        target_grib = pygrib.open(self.target_paths[idx])
        rho_target = torch.from_numpy(np.array(target_grib.select()[0].values, dtype=np.float32)).to(device)
        u_hori_target = torch.from_numpy(np.array(target_grib.select()[1].values, dtype=np.float32)).to(device)
        u_vert_target = torch.from_numpy(np.array(target_grib.select()[2].values, dtype=np.float32)).to(device)
        u_vert, u_hori, rho = self.transform(u_vert, u_hori, rho)
        u_vert_target, u_hori_target, rho_target = self.target_transform(u_vert_target, u_hori_target, rho_target, self.margin)
        return {
            "u_vert": u_vert,
            "u_hori": u_hori,
            "rho": rho,
            "u_vert_target": u_vert_target,
            "u_hori_target": u_hori_target,
            "rho_target": rho_target,
            "datetime": datetime,
        }

In [ ]:
class WeatherDataset(torch.utils.data.Dataset):
    def __init__(self, src_dir, datetimes, transform, target_transform, margin):
        self.src_dir = src_dir
        self.input_paths = []
        self.target_paths = []
        self.datetimes = []
        self.margin = margin # this value is the number of streaming layers that reduce the border cells.
        # eg. ...                            xxx
        #     ...  --(1 Streaming Layer)-->  x.x
        #     ...                            xxx
        for datetime in datetimes:
            self.datetimes.append(datetime.strftime('%Y%m%d%H'))
            self.input_paths.append(os.path.join(src_dir, datetime.strftime('%Y%m%d%H.grib2')))
            datetime += timedelta(hours=3)
            self.target_paths.append(os.path.join(src_dir, datetime.strftime('%Y%m%d%H.grib2')))
        self.rhos = []
        self.u_horis = []
        self.u_verts = []
        self.rho_targets = []
        self.u_hori_targets = []
        self.u_vert_targets = []
        self.transform = transform
        self.target_transform = target_transform
        now = time.time()
        for idx in range(len(self.input_paths)):
            if (idx+1) % 100 == 0:
                print(f'{idx+1} samples collected, {time.time() - now:.2f} s')
                now = time.time()
            input_grib = pygrib.open(self.input_paths[idx])
            rho = torch.from_numpy(np.array(input_grib.select()[0].values, dtype=np.float32)).to(device)
            u_hori = torch.from_numpy(np.array(input_grib.select()[1].values, dtype=np.float32)).to(device)
            u_vert = torch.from_numpy(np.array(input_grib.select()[2].values, dtype=np.float32)).to(device)
            target_grib = pygrib.open(self.target_paths[idx])
            rho_target = torch.from_numpy(np.array(target_grib.select()[0].values, dtype=np.float32)).to(device)
            u_hori_target = torch.from_numpy(np.array(target_grib.select()[1].values, dtype=np.float32)).to(device)
            u_vert_target = torch.from_numpy(np.array(target_grib.select()[2].values, dtype=np.float32)).to(device)
            self.rhos.append(rho)
            self.u_horis.append(u_hori)
            self.u_verts.append(u_vert)
            self.rho_targets.append(rho_target)
            self.u_hori_targets.append(u_hori_target)
            self.u_vert_targets.append(u_vert_target)
    
    def __len__(self):
        return len(self.input_paths)
    
    def __getitem__(self, idx):
        datetime = self.datetimes[idx]
        u_vert, u_hori, rho = self.transform(self.u_verts[idx], self.u_horis[idx], self.rhos[idx])
        u_vert_target, u_hori_target, rho_target = self.target_transform(self.u_vert_targets[idx], self.u_hori_targets[idx], self.rho_targets[idx], self.margin)
        return {
            "u_vert": u_vert,
            "u_hori": u_hori,
            "rho": rho,
            "u_vert_target": u_vert_target,
            "u_hori_target": u_hori_target,
            "rho_target": rho_target,
            "datetime": datetime
        }

In [ ]:
# test for WeatherDataset
def test_for_weatherdataset():
    datetimeset = [dt.datetime(2011, 7, 1), dt.datetime(2011, 7, 2)]
    grib = pygrib.open('/content/drive/MyDrive/lab_data/data/2011070103.grib2')
    u_vert = np.array(grib.select()[2].values, dtype=np.float32)
    src_dir = '/content/drive/MyDrive/lab_data/data/'
    weather_dataset = WeatherDataset(src_dir, datetimeset, transform, target_transform, 1)
    u_vert_direct = torch.from_numpy(u_vert).to(device)
    u_vert_target = weather_dataset.__getitem__(0)['u_vert_target']
    print(weather_dataset.__len__()) # 2
    print(torch.equal(u_vert_target, u_vert_direct[1:-1, 1:-1] * -0.01)) # True
    print(u_vert_direct.size())

test_for_weatherdataset()

2
False
torch.Size([505, 481])


### Dataloader

In [ ]:
def make_dataset_and_dataloader():
    print('Creating a dataset and dataloader...')
    data_dir = '/content/drive/MyDrive/lab_data/data'
    datetimeset = [dt.datetime(2011+i, 7, 1, 0) + timedelta(days=j) for i in range(10) for j in range(92)]
    dataset = WeatherDataset(data_dir, datetimeset, transform, target_transform, LBMwithNoisyInit.margin)
    batch_size = 1
    train_split = 0.8
    shuffle_dataset = True # if false, this model will learn only by the 'past' data, and forcast the 'future' data.
    random_seed = 42
    
    dataset_size =  len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(train_split * dataset_size))
    if shuffle_dataset:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, valid_indices = indices[:split], indices[split:]
    
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(valid_indices)
    
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=0)
    valid_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=0)

    return dataset, train_loader, valid_loader

dataset, train_loader, valid_loader = make_dataset_and_dataloader()

Creating a dataset and dataloader...
100 samples collected, 8.90 s
200 samples collected, 7.99 s
300 samples collected, 5.20 s
400 samples collected, 5.10 s
500 samples collected, 5.13 s
600 samples collected, 5.20 s
700 samples collected, 6.44 s
800 samples collected, 6.21 s
900 samples collected, 5.28 s


# Main

### Loss func

In [ ]:
def loss_func(pred: Dict[str, torch.Tensor], batch: Dict[str, torch.Tensor]):
    # MSE
    u_vert_loss = torch.sum(torch.square(pred['u_vert'] - batch['u_vert_target'].squeeze())) * U_MSE_COEF
    u_hori_loss = torch.sum(torch.square(pred['u_hori'] - batch['u_hori_target'].squeeze())) * U_MSE_COEF
    rho_loss = torch.sum(torch.square(pred['rho'] - batch['rho_target'].squeeze())) * RHO_MSE_COEF

    # MAE
    u_vert_mae = torch.sum(torch.abs(pred['u_vert'] - batch['u_vert_target'].squeeze())) / (U_COEF * HEIGHT * WIDTH) # actual MAE (m/s) per cell
    u_hori_mae = torch.sum(torch.abs(pred['u_hori'] - batch['u_hori_target'].squeeze())) / (U_COEF * HEIGHT * WIDTH)
    rho_mae = torch.sum(torch.abs(pred['rho'] - batch['rho_target'].squeeze())) / (RHO_COEF * HEIGHT * WIDTH * 100.) # hPa

    return {
        'u_vert': u_vert_loss,
        'u_hori': u_hori_loss,
        'rho': rho_loss,
        'total': u_vert_loss + u_hori_loss + rho_loss,
        'u_vert_mae': u_vert_mae,
        'u_hori_mae': u_hori_mae,
        'rho_mae': rho_mae
    }

### Metrics

In [ ]:
class WeightDiffLogger:
    def __init__(self):
        pass

    def memorize(self, lbm: LBMwithNoisyInit):
        self.prev_tensors = {
            'streaming1_w0': lbm.streaming1_layer.w0.clone(),
            'streaming1_w1': lbm.streaming1_layer.w1.clone(),
            'colliding1_w1': lbm.colliding1_layer.w1.clone(),
            'colliding1_w2': lbm.colliding1_layer.w2.clone(),
            'colliding1_w3': lbm.colliding1_layer.w3.clone(),
            'colliding1_w4': lbm.colliding1_layer.w4.clone(),
            'colliding1_w5': lbm.colliding1_layer.w5.clone(),
            'streaming2_w0': lbm.streaming2_layer.w0.clone(),
            'streaming2_w1': lbm.streaming2_layer.w1.clone(),
            'colliding2_w1': lbm.colliding2_layer.w1.clone(),
            'colliding2_w2': lbm.colliding2_layer.w2.clone(),
            'colliding2_w3': lbm.colliding2_layer.w3.clone(),
            'colliding2_w4': lbm.colliding2_layer.w4.clone(),
            'colliding2_w5': lbm.colliding2_layer.w5.clone(),
            'streaming3_w0': lbm.streaming3_layer.w0.clone(),
            'streaming3_w1': lbm.streaming3_layer.w1.clone(),
            'colliding3_w1': lbm.colliding3_layer.w1.clone(),
            'colliding3_w2': lbm.colliding3_layer.w2.clone(),
            'colliding3_w3': lbm.colliding3_layer.w3.clone(),
            'colliding3_w4': lbm.colliding3_layer.w4.clone(),
            'colliding3_w5': lbm.colliding3_layer.w5.clone(),
            'streaming4_w0': lbm.streaming4_layer.w0.clone(),
            'streaming4_w1': lbm.streaming4_layer.w1.clone(),
            'colliding4_w1': lbm.colliding4_layer.w1.clone(),
            'colliding4_w2': lbm.colliding4_layer.w2.clone(),
            'colliding4_w3': lbm.colliding4_layer.w3.clone(),
            'colliding4_w4': lbm.colliding4_layer.w4.clone(),
            'colliding4_w5': lbm.colliding4_layer.w5.clone(),
            'streaming5_w0': lbm.streaming5_layer.w0.clone(),
            'streaming5_w1': lbm.streaming5_layer.w1.clone(),
        }

    def calc_and_log(self, lbm: LBMwithNoisyInit, writer: SummaryWriter, epoch: int):
        with torch.no_grad():
            writer.add_scalars(
                'weight_diff',
                {
                    'streaming1_w0': torch.mean(torch.abs(self.prev_tensors['streaming1_w0'] - lbm.streaming1_layer.w0)),
                    'streaming1_w1': torch.mean(torch.abs(self.prev_tensors['streaming1_w1'] - lbm.streaming1_layer.w1)),
                    'colliding1_w1': torch.mean(torch.abs(self.prev_tensors['colliding1_w1'] - lbm.colliding1_layer.w1)),
                    'colliding1_w2': torch.mean(torch.abs(self.prev_tensors['colliding1_w2'] - lbm.colliding1_layer.w2)),
                    'colliding1_w3': torch.mean(torch.abs(self.prev_tensors['colliding1_w3'] - lbm.colliding1_layer.w3)),
                    'colliding1_w4': torch.mean(torch.abs(self.prev_tensors['colliding1_w4'] - lbm.colliding1_layer.w4)),
                    'colliding1_w5': torch.mean(torch.abs(self.prev_tensors['colliding1_w5'] - lbm.colliding1_layer.w5)),
                    'streaming2_w0': torch.mean(torch.abs(self.prev_tensors['streaming2_w0'] - lbm.streaming2_layer.w0)),
                    'streaming2_w1': torch.mean(torch.abs(self.prev_tensors['streaming2_w1'] - lbm.streaming2_layer.w1)),
                    'colliding2_w1': torch.mean(torch.abs(self.prev_tensors['colliding2_w1'] - lbm.colliding2_layer.w1)),
                    'colliding2_w2': torch.mean(torch.abs(self.prev_tensors['colliding2_w2'] - lbm.colliding2_layer.w2)),
                    'colliding2_w3': torch.mean(torch.abs(self.prev_tensors['colliding2_w3'] - lbm.colliding2_layer.w3)),
                    'colliding2_w4': torch.mean(torch.abs(self.prev_tensors['colliding2_w4'] - lbm.colliding2_layer.w4)),
                    'colliding2_w5': torch.mean(torch.abs(self.prev_tensors['colliding2_w5'] - lbm.colliding2_layer.w5)),
                    'streaming3_w0': torch.mean(torch.abs(self.prev_tensors['streaming3_w0'] - lbm.streaming3_layer.w0)),
                    'streaming3_w1': torch.mean(torch.abs(self.prev_tensors['streaming3_w1'] - lbm.streaming3_layer.w1)),
                    'colliding3_w1': torch.mean(torch.abs(self.prev_tensors['colliding3_w1'] - lbm.colliding3_layer.w1)),
                    'colliding3_w2': torch.mean(torch.abs(self.prev_tensors['colliding3_w2'] - lbm.colliding3_layer.w2)),
                    'colliding3_w3': torch.mean(torch.abs(self.prev_tensors['colliding3_w3'] - lbm.colliding3_layer.w3)),
                    'colliding3_w4': torch.mean(torch.abs(self.prev_tensors['colliding3_w4'] - lbm.colliding3_layer.w4)),
                    'colliding3_w5': torch.mean(torch.abs(self.prev_tensors['colliding3_w5'] - lbm.colliding3_layer.w5)),
                    'streaming4_w0': torch.mean(torch.abs(self.prev_tensors['streaming4_w0'] - lbm.streaming4_layer.w0)),
                    'streaming4_w1': torch.mean(torch.abs(self.prev_tensors['streaming4_w1'] - lbm.streaming4_layer.w1)),
                    'colliding4_w1': torch.mean(torch.abs(self.prev_tensors['colliding4_w1'] - lbm.colliding4_layer.w1)),
                    'colliding4_w2': torch.mean(torch.abs(self.prev_tensors['colliding4_w2'] - lbm.colliding4_layer.w2)),
                    'colliding4_w3': torch.mean(torch.abs(self.prev_tensors['colliding4_w3'] - lbm.colliding4_layer.w3)),
                    'colliding4_w4': torch.mean(torch.abs(self.prev_tensors['colliding4_w4'] - lbm.colliding4_layer.w4)),
                    'colliding4_w5': torch.mean(torch.abs(self.prev_tensors['colliding4_w5'] - lbm.colliding4_layer.w5)),
                    'streaming5_w0': torch.mean(torch.abs(self.prev_tensors['streaming5_w0'] - lbm.streaming5_layer.w0)),
                    'streaming5_w1': torch.mean(torch.abs(self.prev_tensors['streaming5_w1'] - lbm.streaming5_layer.w1)),
                },
                epoch
            )

In [ ]:
def log_typhoon_forcast(pred: Dict[str, torch.Tensor], batch: Dict[str, torch.Tensor], writer: SummaryWriter, epoch: int):
    if batch['datetime'][0] == '2020090200' and (epoch+1) % 5 == 0:
        with torch.no_grad():
            u_vert_in_trimmed, u_hori_in_trimmed, _ = remove_margin(batch['u_vert'].squeeze(), batch['u_hori'].squeeze(), batch['rho'].squeeze(), LBMwithNoisyInit.margin)
            heatmap = torch.sqrt((pred['u_vert'] - batch['u_vert_target'].squeeze()) ** 2.0\
                                + (pred['u_hori'] - batch['u_hori_target'].squeeze()) ** 2.0)\
                    - torch.sqrt((u_vert_in_trimmed - batch['u_vert_target'].squeeze()) ** 2.0\
                                + (u_hori_in_trimmed - batch['u_hori_target'].squeeze()) ** 2.0)
            fig = plt.figure()
            sns.heatmap(heatmap.cpu().detach().numpy(), vmin=-0.15, vmax=0.15, cmap='bwr')
            writer.add_figure(tag='typhoon forcast', figure=fig, global_step=epoch)

In [ ]:
class Loss:
    def __init__(self):
        self.losses_train = {'u_vert': 0.0, 'u_hori': 0.0, 'rho': 0.0, 'total': 0.0, 'u_vert_mae': 0.0, 'u_hori_mae': 0.0, 'rho_mae': 0.0}
        self.losses_val = {'u_vert': 0.0, 'u_hori': 0.0, 'rho': 0.0, 'total': 0.0, 'u_vert_mae': 0.0, 'u_hori_mae': 0.0, 'rho_mae': 0.0}
        self.train_count = 0
        self.val_count = 0

    def add_losses_train(self, losses: Dict[str, torch.Tensor]):
        self.train_count += 1
        for k in self.losses_train.keys():
            self.losses_train[k] += losses[k].item()

    def add_losses_val(self, losses: Dict[str, torch.Tensor]):
        self.val_count += 1
        for k in self.losses_val.keys():
            self.losses_val[k] += losses[k].item()

    def get_losses_train(self):
        return {
            k: self.losses_train[k] / self.train_count for k in self.losses_train.keys()
        }

    def get_losses_val(self):
        return {
            k: self.losses_val[k] / self.val_count for k in self.losses_val.keys()
        }

    def log(self, writer: SummaryWriter, epoch: int, tag: str, keys: List[str], prefix: str = ''):
        d = {}
        losses_train = self.get_losses_train()
        losses_val = self.get_losses_val()
        for k in keys:
            d[f'{prefix}train_{k}'] = losses_train[k]
            d[f'{prefix}test_{k}'] = losses_val[k]

        writer.add_scalars(
            tag,
            d,
            epoch
        )

### calc MAE of datasets

In [ ]:
def calc_losses_between_0h3h(train_loader, valid_loader):
    loss = Loss()
    for (batch_idx, batch) in enumerate(train_loader):
        u_vert_input, u_hori_input, rho_input = remove_margin(
            batch['u_vert'].squeeze(), batch['u_hori'].squeeze(), batch['rho'].squeeze(), LBMwithNoisyInit.margin
        )
        loss_vals = loss_func({
            'u_vert': u_vert_input,
            'u_hori': u_hori_input,
            'rho': rho_input
        }, batch)
        loss.add_losses_train(loss_vals)

    for (batch_idx, batch) in enumerate(valid_loader):
        u_vert_input, u_hori_input, rho_input = remove_margin(
            batch['u_vert'].squeeze(), batch['u_hori'].squeeze(), batch['rho'].squeeze(), LBMwithNoisyInit.margin
        )
        loss_vals = loss_func({
            'u_vert': u_vert_input,
            'u_hori': u_hori_input,
            'rho': rho_input
        }, batch)
        loss.add_losses_val(loss_vals)

    return loss

### Main

In [ ]:
# todo: enable the model to receive batch data
exec_datetime = dt.datetime.now()
log_dir = f'/content/drive/MyDrive/lab_data/logs/{exec_datetime.strftime("%Y%m%d_%H%M%S")}'
writer = SummaryWriter(log_dir=log_dir)

def main(dataset, train_loader, valid_loader, writer: SummaryWriter):
    print('Calc losses between 0h and 3h...')
    losses_0h3h = calc_losses_between_0h3h(train_loader, valid_loader)
    print(f'train u_vert mae: {losses_0h3h.get_losses_train()["u_vert_mae"]} m/s')
    print(f'train u_hori mae: {losses_0h3h.get_losses_train()["u_hori_mae"]} m/s')
    print(f'train rho mae: {losses_0h3h.get_losses_train()["rho_mae"]} hPa')
    print(f'val u_vert mae: {losses_0h3h.get_losses_val()["u_vert_mae"]} m/s')
    print(f'val u_hori mae: {losses_0h3h.get_losses_val()["u_hori_mae"]} m/s')
    print(f'val rho mae: {losses_0h3h.get_losses_val()["rho_mae"]} hPa')
    print('Creating the model...')
    lbm = LBMwithNoisyInit(HEIGHT, WIDTH).to(device)
    weight_diff_logger = WeightDiffLogger()

    num_epochs = 350
    total_time = 0.0
    

    optimizer = torch.optim.Adam(lbm.parameters())
    # optimizer = torch.optim.Adadelta(lbm.parameters())
    # optimizer = torch.optim.AdamW(lbm.parameters())
    # optimizer = torch.optim.SGD(lbm.parameters(), lr=0.001, momentum=0.9, nesterov=True)

    
    print("optimizer = " + str(optimizer))
    print("max_epochs = %3d " % num_epochs)
    print("summary writer's logdir = " + writer.log_dir )
    
    print('Training the model...')
    
    for epoch in range(num_epochs):
        lbm.train()
        epoch_loss = Loss()
        epoch_started_time = time.time()
        weight_diff_logger.memorize(lbm)
        for (batch_idx, batch) in enumerate(train_loader):
            optimizer.zero_grad()
            out = lbm(batch['u_vert'].squeeze(), batch['u_hori'].squeeze(), batch['rho'].squeeze())
            loss_vals = loss_func(out, batch)
            epoch_loss.add_losses_train(loss_vals)
            log_typhoon_forcast(out, batch, writer, epoch)
            loss_vals['total'].backward()
            optimizer.step()
        
        lbm.eval()
        with torch.no_grad():
            for (batch_idx, batch) in enumerate(valid_loader):
                out = lbm(batch['u_vert'].squeeze(), batch['u_hori'].squeeze(), batch['rho'].squeeze())
                loss_vals = loss_func(out, batch)
                epoch_loss.add_losses_val(loss_vals)
                log_typhoon_forcast(out, batch, writer, epoch)

        losses_train = epoch_loss.get_losses_train()
        losses_val = epoch_loss.get_losses_val()
        weight_diff_logger.calc_and_log(lbm, writer, epoch)
        print("epoch = %4d  train_loss = %.4e  val_loss = %.4e  u_vert_loss = %.4e  u_hori_loss = %.4e  rho_loss = %.4e  epoch time = %0.2fs"\
              % (epoch, losses_train['total'], losses_val['total'], losses_val['u_vert'], losses_val['u_hori'], losses_val['rho'], time.time() - epoch_started_time))
        epoch_loss.log(writer, epoch, 'MSE_loss', ['total', 'u_vert', 'u_hori', 'rho'])
        # epoch_loss.log(writer, epoch, 'MAE', ['u_vert_mae', 'u_hori_mae', 'rho_mae'])
        # losses_0h3h.log(writer, epoch, 'MAE', ['u_vert_mae', 'u_hori_mae', 'rho_mae'], prefix='0h3h')


    print('Done')
        
main(dataset, train_loader, valid_loader, writer)

Calc losses between 0h and 3h...
train u_vert mae: 1.1112427019878575 m/s
train u_hori mae: 1.0288740297698455 m/s
train rho mae: 0.7113683552521727 hPa
val u_vert mae: 1.0980090366109558 m/s
val u_hori mae: 1.0207934680840243 m/s
val rho mae: 0.7064201222813647 hPa
Creating the model...
optimizer = Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
max_epochs = 350 
summary writer's logdir = /content/drive/MyDrive/lab_data/logs/20221230_080752
Training the model...
epoch =    0  train_loss = 1.2670e+06  val_loss = 1.2184e+06  u_vert_loss = 5.7380e+05  u_hori_loss = 5.1381e+05  rho_loss = 1.3075e+05  epoch time = 45.66s
epoch =    1  train_loss = 1.2343e+06  val_loss = 1.2133e+06  u_vert_loss = 5.7025e+05  u_hori_loss = 5.0970e+05  rho_loss = 1.3340e+05  epoch time = 44.74s
epoch =    2  train_loss = 1.2232e+06  val_l

# Tensorboard

In [ ]:
%tensorboard --logdir=$log_dir
# %tensorboard --logdir=/content/drive/MyDrive/lab_data/logs/20221229_165644

<IPython.core.display.Javascript object>

# Research Diary

## 2022/12/25


### したこと
- 層をもう一層ふやして、streamingを5層、collidingを4層にした
  - 若干精度良くなった気がする？その前に6.5層でためしたら収束しなくなってしまったので、層を減らしてみた
- なんか別に勾配消失がおこっているわけでもなく、むしろ勾配爆発しているようなきもする？
  - optimizerをAdams(lrなどはデフォルト値)とかでやってみた結果
    - <img src="https://drive.google.com/uc?id=1egCA3TmOssUKLbXI39WgBf77S0KxHdOC" width='400px'>
- なので、この後に勾配爆発しないような対策を考える必要がある
  1. 勾配を制限する
  1. u_vert, u_hori, rhoそれぞれの係数を再度検討する。きちんと数値的に分析して係数変化させてみたほうが良さそう。
    - それから、係数がマジックナンバーになっているので、定数化したほうがよいだろう
  1. [parameterごとにlrが設定できるらしい](https://pystyle.info/pytorch-sgd/#:~:text=17-,%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%81%94%E3%81%A8%E3%81%AB%E5%90%84%E7%A8%AE%E4%BF%82%E6%95%B0%E3%82%92%E5%A4%89%E6%9B%B4%E3%81%97%E3%81%9F%E3%81%84%E5%A0%B4%E5%90%88,-torch.optim.SGD)
- 現在Adamsでやっているが他のoptimizerも使えないか検証している
- リファクタリングしたほうがいい すでにだいぶ読みにくくなっている
- なぜか台風の日の画像がすべて取得できていないんだがなぜ…？

### 今後の優先順位

1. optimizerを変えたときの比較
1. リファクタリング
  1. u_vert, u_hori, rhoの係数を定数化する
  1. main関数のメトリクスを取るところを関数化
  1. u_vert, u_horiのMAEをとって、三時間でのMAEも取っておく(グラフで定数値として表しておくとわかりやすいと思う), それからu_squaredとrhoのlossもメトリクスを取っておいたほうがバランスよく学習できているか見やすい気がする
  1. 台風の日の画像がすべて取れるようにデバッグ
  1. モデルを100epochごとに保存
1. 勾配爆発の対策
1. パラメータを増やす 特に、(feq + fprev) / 2.0のところをパラメータ化して学習させてみる

### 結果
1. optimizerを変えたときの画像比較
  1. Adam ... 最後に勾配爆発しているが、概ねよく収束している
    - <img src="https://drive.google.com/uc?id=14VJY4AnRNW-LXsWX3E7q_urOCADuQPxX" width='200px'> <img src="https://drive.google.com/uc?id=1xdePf_J_wcNOAT_GCm2tdmSBnwCiKJzU" width='200px'>
  1. Adadelta ... ダメそう 早々に勾配爆発してしまっているし、収束も悪かった
    - <img src="https://drive.google.com/uc?id=133r10FBEA7hA-fELX_FJofO_dXc7eV6A" width='200px'> <img src="https://drive.google.com/uc?id=1ezzPaYjngrJ1VC_R59k_NQIh64zL3Jze" width='200px'>
  1. AdamW ... だめっぽい。途中から勾配爆発してしまった
    - <img src="https://drive.google.com/uc?id=1YTnioqIen_FujQEm8DZIDaq19CunWzh2" width='200px'> <img src="https://drive.google.com/uc?id=1X3kU2nv-EtGdvBbDenXyxnLkVXzZFm4I" width='200px'>
  1. NAG ... なぜかバグってできない...

1. リファクタリング
  1. u_vert, u_hori, rhoの係数を定数化
  1. weightのメトリクスとるところを関数化した
  1. 台風の画像取るところを関数化した
  1. lossもクラス化して若干みえやすくなった…と思う
  1. MAEもとれるようにした。メトリクスが充実してきた感がある
  1. 0hと3hのMAEをとって比較できるようにした
  1. modelのほぞんは次回

rho のMAEがめちゃでかいのがきになる なぜ？



## 進捗ゼミ 2022/12/21

### 今回したこと
- ここにあるコード全部
- PyTorchに移植が完了した！👏
  - GPUで速度が爆速に 2層で1epochあたり1時間かかっていたのが、4層まで増やして1epochあたり30秒で終わるようになった
  - もう逆誤差伝播を手計算する必要がない(自動微分)
  - SGD, Adamなどの最適化関数が用意されているので、それを使うだけ
- いままでは出力のu_vert, u_horiしか損失関数に使っていなかったが、その他にもrhoを使うようにした

### これからやること

- もう少しepoch数を増やして実験(?)
- Adam以外の最適化関数の利用、比較検討
- 学習済みモデルを都度保存する
  - モデルのインスタンスがmain関数に埋め込まれているので、これを外に出す
- 重み成分のチェック（勾配が消失していないか確かめる）
  - 勾配が消失していないなら層を増やしてみる
  - 消失しているなら重みの伝え方をもう少し検討
    - Resnetのように入力値を再度足し合わせる
    - 線形補間した教師データを用意し、転移学習
      - 0時間、3時間、6時間でどれくらい線形的な関係にあるのかを調べる必要がありそう
- InputLayerの重みも学習できるようにする
- ほかのデータ(たとえば海流など)も使えないか考えてみる